INSTALLATIONS & IMPORTS

In [ ]:
!pip install pyGMs

import pyGM as gm 
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline  

     |████████████████████████████████| 163kB 2.9MB/s 


LOAD DATA FUNCTION

In [ ]:
def load_data(filename, pKeep, nEdge, nKeep):

  # Load training data and reduce (subsample) if desired

  # Read thru file to get numeric ids for each player 
  with open(filename) as f: lines = f.read().split('\n')

  p = 0; playerid = {};
  for i in range(len(lines)):
    csv = lines[i].split(',');
    if len(csv) != 10: continue;   # parse error or blank line
    player0,player1 = csv[1],csv[4];
    if player0 not in playerid: playerid[player0]=p; p+=1;
    if player1 not in playerid: playerid[player1]=p; p+=1;

  nplayers = len(playerid)
  playername = ['']*nplayers
  for player in playerid: playername[ playerid[player] ]=player;  # id to name lookup


  # Sparsifying parameters (discard some training examples):
  pKeep = pKeep   # fraction of edges to consider (immed. throw out 1-p edges)
  nEdge = nEdge     # try to keep nEdge opponents per player (may be more; asymmetric)
  nKeep = nKeep     # keep at most nKeep games per opponent pairs (play each other multiple times)

  nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) );
  for i in range(len(lines)):
    csv = lines[i].split(',');
    if len(csv) != 10: continue;   # parse error or blank line
    a,b = playerid[csv[1]],playerid[csv[4]];
    aw,bw = csv[2]=='[winner]',csv[5]=='[winner]';
    if (np.random.rand() < pKeep):
      if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
        nplays[a,b] += 1; nplays[b,a]+=1; nwins[a,b] += aw; nwins[b,a] += bw;

  return (nplayers, nplays, nwins)

GENERATE GAME OUTCOMES FUNCTION

In [ ]:
def generate_games(nwins):
  games = []

  for player in range(len(nwins)):
    for opponent in range(len(nwins[player])):
      for i in range(int(nwins[player][opponent])):
        games.append((player, opponent, +1))
        games.append((opponent, player, -1))

  return games

CREATE MODEL FUNCTION

In [ ]:
def create_model(nlevels, scale, games, nplayers):  
  nlevels = nlevels   # let's say 10 discrete skill levels
  scale = scale     # this scales how skill difference translates to win probability

  games = games
  nplayers = nplayers

  # Make variables for each player; value = skill level
  X = [None]*nplayers
  for i in range(nplayers):
      X[i] = gm.Var(i, nlevels)   

  # Information from each game: what does Pi winning over Pj tell us?
  #    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
  Pwin = np.zeros( (nlevels,nlevels) )
  for i in range(nlevels):
      for j in range(nlevels):
          diff = i-j                   # find the advantage of Pi over Pj, then 
          Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

  # before any games, uniform belief over skill levels for each player:
  factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

  # Now add the information from each game:
  for g in games:
      P1,P2,win = g[0],g[1],g[2]
      if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
      factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

  model = gm.GraphModel(factors)
  model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

  if model.nvar < 0:       # for very small models, we can do brute force inference:
      jt = model.joint()
      jt /= jt.sum()       # normalize the distribution and marginalize the table
      bel = [jt.marginal([i]) for i in range(nplayers)] 
  else:                    # otherwise we need to use some approximate inference:
      from pyGM.messagepass import LBP, NMF
      #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
      lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

  return bel, X, Pwin

WIN PROBABILITY FUNCTION

In [ ]:
# Expected value (over skill of P0, P1) of Pr[win | P0-P1]

def win_probability(i, j, bel, X, Pwin):
  if i<j:
      return ( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() )
  else:
      return ( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum() )

    
# Notes: we should probably use the joint belief over Xi and Xj, but for simplicity
#  with approximate inference we'll just use the estimated singleton marginals

GENERATE WINRATES FUNCTION

In [ ]:
# Establish winrate for each pair of players

def generate_winrates(all_games):
  wins = dict() # wins[i, j] is the number of times that i has beaten j

  for g in all_games:
    if (g[0], g[1]) not in wins and g[2] == +1:
      wins[(g[0], g[1])] = 1
    elif (g[0], g[1]) in wins and g[2] == +1:
      wins[(g[0], g[1])] += 1

  winrates = dict() # winrate[i, j] is the percent of the time that i has beaten j

  for w in wins:
    if (w[1], w[0]) not in wins:
      winrates[w] = 1.0
    else:
      winrates[w] = wins[w] / (wins[w] + wins[(w[1], w[0])])

  return winrates

GENERATE MSE FUNCTION

In [ ]:
def generate_mse(winrates, bel, X, Pwin):
  total_squared_error = 0.0

  for w in winrates:
    total_squared_error += ((winrates[w] - win_probability(w[0], w[1], bel, X, Pwin)) ** 2.0)

  return total_squared_error / len(winrates)

SKILL PREDICTION ACCURACY FUNCTION

In [ ]:
def generate_skill_prediction_accuracy(winrates, all_skill_levels):

  accurate = 0

  total = 0

  for w in winrates:
    if (winrates[w] > 0.5):
      if all_skill_levels[w[0]] > all_skill_levels[w[1]]:
        accurate += 1
      total += 1

    elif (winrates[w] < 0.5):
      if all_skill_levels[w[0]] < all_skill_levels[w[1]]:
        accurate += 1
      total += 1

    else:
      pass

  return (accurate / total)

RUN TEST FUNCTION

In [ ]:
def run_test(filename, nlevels, scale, bel, X, Pwin, pKeep, nEdge, nKeep):
  nplayers, nplays, nwins = load_data(filename, pKeep, nEdge, nKeep)
  games = generate_games(nwins)
  all_skill_levels = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]
  winrates = generate_winrates(games)
  mse = (generate_mse(winrates, bel, X, Pwin))
  skill_prediction_accuracy = (generate_skill_prediction_accuracy(winrates, all_skill_levels))
  return (mse, skill_prediction_accuracy)

CREATE AND RUN MODEL FUNCTION

In [ ]:
def create_and_run_model(train_file, valid_file, nlevels, scale, pKeep_train, nEdge_train, nKeep_train, pKeep_valid,
                         nEdge_valid, nKeep_valid):
  nplayers, nplays, nwins = load_data(train_file, pKeep_train, nEdge_train, nKeep_train)
  bel, X, Pwin = create_model(nlevels, scale, generate_games(nwins), nplayers)

  return (run_test(valid_file, nlevels, scale, bel, X, Pwin, pKeep_valid, nEdge_valid, nKeep_valid),
          run_test(train_file, nlevels, scale, bel, X, Pwin, pKeep_train, nEdge_train, nKeep_train))

TESTING DIFFERENT MODELS

In [ ]:
'''scale_model_results = []

for i in range(11):
  scale_model_results.append(create_and_run_model("train.csv", "valid.csv", 10, (i / 10.0), 1.0, 3, 5, 1.0, 3, 5))'''

'scale_model_results = []\n\nfor i in range(11):\n  scale_model_results.append(create_and_run_model("train.csv", "valid.csv", 10, (i / 10.0), 1.0, 3, 5, 1.0, 3, 5))'

In [ ]:
scale_model_results = [((0.2040047338067142, 0.0), (0.2056532503596645, 0.0)), ((0.19646618736514826, 0.5985308208240179), (0.13250049221907087, 0.8654895666131621)), ((0.19790314807931988, 0.6014053018205047), (0.09056222649870875, 0.8709470304975923)), ((0.20822886247047812, 0.6058767167039285), (0.07453881851107211, 0.8776886035313002)), ((0.21866964217811216, 0.6030022357074417), (0.06741191011641443, 0.8828250401284109)), ((0.2278788900294019, 0.6023634621526669), (0.06380156820968169, 0.8866773675762439)), ((0.23577497045610749, 0.6045991695943788), (0.06186637252102992, 0.8863563402889245)), ((0.2389916620390329, 0.6167358671351006), (0.05939435353751756, 0.8969502407704655)), ((0.24958126967191438, 0.594059405940594), (0.05923099788762621, 0.8975922953451043)), ((0.25429833257437934, 0.5937400191632066), (0.058824369234982944, 0.8979133226324237)), ((0.2582585995398611, 0.5911849249441073), (0.05866716068361593, 0.8966292134831461))]

for i in range(len(scale_model_results)):
  print("----------")
  print("nlevels 10")
  print("scale", i / 10.0)
  print("Train MSE", scale_model_results[i][1][0])
  print("Train skill prediction accuracy", scale_model_results[i][1][1])
  print("Valid MSE", scale_model_results[i][0][0])
  print("Valid skill prediction accuracy", scale_model_results[i][0][1])
  print("----------")

----------
nlevels 10
scale 0.0
Train MSE 0.2056532503596645
Train skill prediction accuracy 0.0
Valid MSE 0.2040047338067142
Valid skill prediction accuracy 0.0
----------
----------
nlevels 10
scale 0.1
Train MSE 0.13250049221907087
Train skill prediction accuracy 0.8654895666131621
Valid MSE 0.19646618736514826
Valid skill prediction accuracy 0.5985308208240179
----------
----------
nlevels 10
scale 0.2
Train MSE 0.09056222649870875
Train skill prediction accuracy 0.8709470304975923
Valid MSE 0.19790314807931988
Valid skill prediction accuracy 0.6014053018205047
----------
----------
nlevels 10
scale 0.3
Train MSE 0.07453881851107211
Train skill prediction accuracy 0.8776886035313002
Valid MSE 0.20822886247047812
Valid skill prediction accuracy 0.6058767167039285
----------
----------
nlevels 10
scale 0.4
Train MSE 0.06741191011641443
Train skill prediction accuracy 0.8828250401284109
Valid MSE 0.21866964217811216
Valid skill prediction accuracy 0.6030022357074417
----------
-------

In [ ]:
'''nlevels_model_results = []

i = 1
while (i < 52):
  nlevels_model_results.append(create_and_run_model("train.csv", "valid.csv", i, 0.7, 1.0, 3, 5, 1.0, 3, 5))
  i += 2'''

'nlevels_model_results = []\n\ni = 1\nwhile (i < 52):\n  nlevels_model_results.append(create_and_run_model("train.csv", "valid.csv", i, 0.7, 1.0, 3, 5, 1.0, 3, 5))\n  i += 2'

In [ ]:
nlevels_model_results = [((0.20400473380671422, 0.0), (0.2056532503596645, 0.0)), ((0.1988661106304083, 0.6020440753752795), (0.09580877721244141, 0.8645264847512039)), ((0.21057271724421248, 0.6189715745768125), (0.07164512725218587, 0.8719101123595505)), ((0.229892978969809, 0.596295113382306), (0.06447787116589113, 0.8825040128410915)), ((0.2388149228444991, 0.6023634621526669), (0.06157804960294816, 0.8853932584269663)), ((0.243116206942469, 0.6141807729160013), (0.05854526751000141, 0.8982343499197432)), ((0.2505349051327576, 0.6103481315873522), (0.05753777102929147, 0.9004815409309791)), ((0.253566600947636, 0.6100287448099648), (0.0567999596128552, 0.9043338683788122)), ((0.25662896208514957, 0.6090705844778026), (0.0565831803054724, 0.9040128410914928)), ((0.2586346409480927, 0.6084318109230278), (0.05620019846344637, 0.9001605136436597)), ((0.26112105352346304, 0.6097093580325774), (0.056224941799965365, 0.9052969502407705)), ((0.2623508093978641, 0.6090705844778026), (0.056155733907939674, 0.9030497592295346)), ((0.26388097755830564, 0.6103481315873522), (0.055988692692918014, 0.9040128410914928)), ((0.26516846275146905, 0.6103481315873522), (0.05602909630294836, 0.9046548956661317)), ((0.2672434176921395, 0.6106675183647396), (0.05609746935526419, 0.8991974317817014)), ((0.26712101709870306, 0.6109869051421272), (0.055936660918596066, 0.9046548956661317)), ((0.26876852868167966, 0.60938997125519), (0.05591498183586237, 0.9011235955056179)), ((0.27096401012410665, 0.6058767167039285), (0.05612947751531459, 0.8972712680577849)), ((0.27014530135289544, 0.60938997125519), (0.05586208351235634, 0.9011235955056179)), ((0.27212085697644395, 0.6068348770360907), (0.056046572932330924, 0.8972712680577849)), ((0.2710274336024684, 0.6106675183647396), (0.055788719200164864, 0.9008025682182985)), ((0.2729809186610065, 0.6081124241456404), (0.05592233073324191, 0.8966292134831461)), ((0.2710750534223711, 0.6119450654742894), (0.055785168170613395, 0.903370786516854)), ((0.27393849101134315, 0.6071542638134781), (0.05596818446156647, 0.8972712680577849)), ((0.271756795713086, 0.611625678696902), (0.0557506238633067, 0.9036918138041734)), ((0.27301984434598286, 0.6113062919195146), (0.05574540840073132, 0.9011235955056179))]

for i in range(len(nlevels_model_results)):
  print("----------")
  print("nlevels", (i * 2) + 1)
  print("scale", 0.7)
  print("Train MSE", nlevels_model_results[i][1][0])
  print("Train skill prediction accuracy", nlevels_model_results[i][1][1])
  print("Valid MSE", nlevels_model_results[i][0][0])
  print("Valid skill prediction accuracy", nlevels_model_results[i][0][1])
  print("----------")

----------
nlevels 1
scale 0.7
Train MSE 0.2056532503596645
Train skill prediction accuracy 0.0
Valid MSE 0.20400473380671422
Valid skill prediction accuracy 0.0
----------
----------
nlevels 3
scale 0.7
Train MSE 0.09580877721244141
Train skill prediction accuracy 0.8645264847512039
Valid MSE 0.1988661106304083
Valid skill prediction accuracy 0.6020440753752795
----------
----------
nlevels 5
scale 0.7
Train MSE 0.07164512725218587
Train skill prediction accuracy 0.8719101123595505
Valid MSE 0.21057271724421248
Valid skill prediction accuracy 0.6189715745768125
----------
----------
nlevels 7
scale 0.7
Train MSE 0.06447787116589113
Train skill prediction accuracy 0.8825040128410915
Valid MSE 0.229892978969809
Valid skill prediction accuracy 0.596295113382306
----------
----------
nlevels 9
scale 0.7
Train MSE 0.06157804960294816
Train skill prediction accuracy 0.8853932584269663
Valid MSE 0.2388149228444991
Valid skill prediction accuracy 0.6023634621526669
----------
----------
nleve

In [ ]:
print("The best model uses nlevels = 5 and scale = 0.7.")

The best model uses nlevels = 5 and scale = 0.7.


COMPARING BEST MODEL TO SIMPLER MODELS

In [ ]:
train_winrates = generate_winrates(generate_games(load_data("train.csv", 1.0, 3, 5)[2]))

valid_winrates = generate_winrates(generate_games(load_data("valid.csv", 1.0, 3, 5)[2]))

In [ ]:
correct_predictions = 0
total_predictions = 0

for k in valid_winrates:
  if k in train_winrates:
    if valid_winrates[k] > 0.5 and train_winrates[k] > 0.5:
      correct_predictions += 1
    if valid_winrates[k] < 0.5 and train_winrates[k] < 0.5:
      correct_predictions += 1
    if valid_winrates[k] == 0.5 and train_winrates[k] == 0.5:
      correct_predictions += 1
    total_predictions += 1

In [ ]:
print("Simple model game prediction accuracy (based on winrate) (validation data)", correct_predictions / total_predictions)
print("Best model game prediction accuracy (based on skill rank) (validation data)", nlevels_model_results[2][0][1])

Simple model game prediction accuracy (based on winrate) (validation data) 0.5985401459854015
Best model game prediction accuracy (based on skill rank) (validation data) 0.6189715745768125


In [ ]:
train_nplays = load_data("train.csv", 1.0, 3, 5)[1]

In [ ]:
correct_predictions = 0
total_predictions = 0

for k in valid_winrates:
  if valid_winrates[k] > 0.5 and np.sum(train_nplays[k[0]]) > np.sum(train_nplays[k[1]]):
    correct_predictions += 1
  if valid_winrates[k] < 0.5 and np.sum(train_nplays[k[0]]) < np.sum(train_nplays[k[1]]):
    correct_predictions += 1
  if valid_winrates[k] == 0.5 and np.sum(train_nplays[k[0]]) == np.sum(train_nplays[k[1]]):
    correct_predictions += 1
  total_predictions += 1

In [ ]:
print("Simple model game prediction accuracy (based on games played) (validation data)", correct_predictions / total_predictions)
print("Best model game prediction accuracy (based on skill rank) (validation data)", nlevels_model_results[2][0][1])

Simple model game prediction accuracy (based on games played) (validation data) 0.4746474647464746
Best model game prediction accuracy (based on skill rank) (validation data) 0.6189715745768125


In [ ]:
train_nwins = load_data("train.csv", 1.0, 3, 5)[2]

In [ ]:
correct_predictions = 0
total_predictions = 0

for k in valid_winrates:
  wr_0 = np.sum(train_nwins[k[0]]) / np.sum(train_nplays[k[0]])
  wr_1 = np.sum(train_nwins[k[1]]) / np.sum(train_nplays[k[1]])

  if valid_winrates[k] > 0.5 and wr_0 > wr_1:
    correct_predictions += 1
  if valid_winrates[k] < 0.5 and wr_0 < wr_1:
    correct_predictions += 1
  if valid_winrates[k] == 0.5 and wr_0 == wr_1:
    correct_predictions += 1
  total_predictions += 1

In [ ]:
print("Simple model game prediction accuracy (based on *total* winrate) (validation data)", correct_predictions / total_predictions)
print("Best model game prediction accuracy (based on skill rank) (validation data)", nlevels_model_results[2][0][1])

Simple model game prediction accuracy (based on *total* winrate) (validation data) 0.46294629462946296
Best model game prediction accuracy (based on skill rank) (validation data) 0.6189715745768125
